Alexis Pérez-Bellido, 2023-05-06 (Tested on python 3.8)

More information about the Tobii SDK can be found at:
https://developer.tobiipro.com/python/python-step-by-step-guide.html

__Install tobii SDK libraries__

In [ ]:
pip install -U pip setuptools # Linux or Mac OS X
python -m pip install -U pip setuptools # Windows

pip install tobii_research

Calibration:
I use the application Tobii Pro Eye tracker manager

https://www.tobii.com/products/software/applications-and-developer-kits/tobii-pro-eye-tracker-manager

Basic libraries to work with tobii in python

In [ ]:
import tobii_research as tr
import time

Find all connected eye trackers

In [ ]:
found_eyetrackers = tr.find_all_eyetrackers()
my_eyetracker = found_eyetrackers[0]
print("Address: " + my_eyetracker.address)
print("Model: " + my_eyetracker.model)
print("Name (It's OK if this is empty): " + my_eyetracker.device_name)
print("Serial number: " + my_eyetracker.serial_number)


__Define a callback function:__

The eye tracker outputs a gaze data sample at a regular interval (30, 60, 120, 300, etc, times per seconds, depending on model). To get hold of this data, you tell the Tobii Pro SDK that you want to subscribe to the gaze data, and then provide the SDK with what is known as a callback function. The callback function is a function like any other, with the exception that you never need to call it yourself; instead it gets called every time there is a new gaze data sample. So, in this callback function, you do whatever it is that you want to do with the gaze data, for example printing some parts of it.

__This is the key part of the code. Here you decide what you want to do with the gaze data__


In [3]:
# this function append all the new gaze data to this list since the last callback
global_gaze_data = [] 
def gaze_data_callback(gaze_data):
    global global_gaze_data
    global_gaze_data.append(gaze_data)

# I am using this one in which I first in the script I initialize a dictionary containing the different gaze variables that I want to collect



global_gaze_data = {}
global_gaze_data['left_pos'] = np.array([np.nan,np.nan])
global_gaze_data['right_pos'] = np.array([np.nan,np.nan])
global_gaze_data['left_pupil'] = np.nan
global_gaze_data['right_pupil'] = np.nan
global_gaze_data['left_pupil_validity'] = np.nan
global_gaze_data['right_pupil_validity'] = np.nan
global_gaze_data['device_time'] = np.nan
global_gaze_data['system_time'] = np.nan

# and then I update the values of the dictionary in the callback function

def gaze_data_callback(gaze_data): # this is a callback function that will return values
    global global_gaze_data
    global_gaze_data['left_pos'] = gaze_data['left_gaze_point_on_display_area']
    global_gaze_data['right_pos'] = gaze_data['right_gaze_point_on_display_area']
    global_gaze_data['left_pupil'] = gaze_data['left_pupil_diameter']
    global_gaze_data['right_pupil'] = gaze_data['right_pupil_diameter']
    global_gaze_data['left_pupil_validity'] = gaze_data['left_pupil_validity']
    global_gaze_data['right_pupil_validity'] = gaze_data['right_pupil_validity']
    global_gaze_data['device_time'] = gaze_data['device_time_stamp']
    global_gaze_data['system_time'] = gaze_data['system_time_stamp']

These are different data variables names that you can get from the eyetracker. You can find more information about them here: https://developer.tobii.com/python/getting-started/

In [ ]:
'''
columns = ['device_time_stamp', 'system_time_stamp', 'left_gaze_direction_unit_vector', 
           'left_gaze_direction_validity', 'left_gaze_origin_position_in_hmd_coordinates', 
           'left_gaze_origin_validity', 'left_pupil_diameter', 'left_pupil_validity', 
           'left_pupil_position_in_tracking_area', 'left_pupil_position_validity', 
           'right_gaze_direction_unit_vector', 'right_gaze_direction_validity', 
           'right_gaze_origin_position_in_hmd_coordinates', 'right_gaze_origin_validity',
           'right_pupil_diameter', 'right_pupil_validity', 'right_pupil_position_in_tracking_area', 
           'right_pupil_position_validity']
'''

"\ncolumns = ['device_time_stamp', 'system_time_stamp', 'left_gaze_direction_unit_vector', \n           'left_gaze_direction_validity', 'left_gaze_origin_position_in_hmd_coordinates', \n           'left_gaze_origin_validity', 'left_pupil_diameter', 'left_pupil_validity', \n           'left_pupil_position_in_tracking_area', 'left_pupil_position_validity', \n           'right_gaze_direction_unit_vector', 'right_gaze_direction_validity', \n           'right_gaze_origin_position_in_hmd_coordinates', 'right_gaze_origin_validity',\n           'right_pupil_diameter', 'right_pupil_validity', 'right_pupil_position_in_tracking_area', \n           'right_pupil_position_validity']\n"

Finally to have to suscribe to the callback function to initiate the eyetracker. 

In [ ]:
my_eyetracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, gaze_data_callback, as_dictionary=True)

This function internally will collect gaze data, but to acccess the gaze data, you will need to call the gaze_data_callback function in your code whenever you want to use the gaze data

In [ ]:
global_gaze_data # this is what you need to call. What I do is to combine the 2 eyes data in a single variable with the following function

# eyes coordinates are relative to the screen size. e.g. (0.5,0.5) is the center of the screen. (1,1) is the top right corner of the screen. 
# You can transform this coordinates to pixels if you know the screen size and resolution.

I s# function to combine eye and left eye data
    def get_combined_eyes(gdata):
        combined_eyes = {}
        LPos = np.array(gdata['left_pos'])
        RPos = np.array(gdata['right_pos'])
        combined_eyes['EyesPos'] = np.nanmean([LPos,RPos], axis = 0)

        LPup = np.array(gdata['left_pupil'])
        RPup = np.array(gdata['right_pupil'])
        combined_eyes['EyesPup'] = np.nanmean([LPup,RPup], axis = 0)  
        return combined_eyes

One example using my functions in combination..

In [ ]:
# The while loop does not stop until the eyes are in the center of the screen.

EyesPos = np.array([0.0,0.0]) # initialising eye position    
eye_lim = 0.65

tobii = True
if tobii:
    eyepos = []
    while EyesPos[0] >  eye_lim or EyesPos[0] < 1-eye_lim or EyesPos[1] >  eye_lim or EyesPos[1] < 1-eye_lim:
    # gdata = global_gaze_data
        Eyes =  get_combined_eyes(global_gaze_data)
        eyepos.append(Eyes['EyesPos']) #
        if len(eyepos) > 500:
            lastpos = eyepos[-30:]
            EyesPos = np.nanmean(lastpos, axis=0)
            #print(EyesPos) # with this you can visualize the eye position
            if np.isnan(EyesPos[0]):  EyesPos = np.array([0.0,0.0])

Finally, you stop gaze data collection and close the connection with the eyetracker by unsubscribing from the gaze data

In [ ]:
my_eyetracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, gaze_data_callback)
